## 1. LLM Chain

In [3]:
from langchain_community.chat_models import ChatZhipuAI

import os
from dotenv import load_dotenv
from zhipuai import ZhipuAI
load_dotenv()

api_key=os.environ.get('ZHIPU_API_KEY')


In [9]:
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "What is {topic}")
])

chat = ChatZhipuAI(
    zhipuai_api_key = api_key,
    model="glm-4",
    max_tokens=1024,
    temperature=0.8, 
)

chain = prompt_template | chat

In [12]:
chain.invoke({"topic":"Peking duck"})

AIMessage(content='北京烤鸭（Peking duck）是一道享誉中外的中华传统名菜，源于中国北京，具有悠久的历史。它的制作工艺独特，食用时通常将烤制好的鸭子片成薄片，与薄饼、甜面酱和葱丝一起卷起食用。\n\n北京烤鸭的特点是鸭皮酥脆，肉质鲜嫩。在烹饪过程中，师傅会在鸭腔内注入热水，使鸭皮与鸭肉之间形成一层薄薄的脂肪，这样在烤制时，脂肪会溶入皮中，使得鸭皮既酥又脆。传统上，北京烤鸭采用挂炉或焖炉的方式烤制，过程中还会不断地在鸭皮上涂刷蜂蜜或糖水，增加其色泽和风味。\n\n北京烤鸭不仅在中国深受欢迎，也深受世界各地美食爱好者的喜爱，成为国际友人了解中国文化的一个重要窗口。', response_metadata={'token_usage': {'completion_tokens': 179, 'prompt_tokens': 18, 'total_tokens': 197}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-d2ac2bec-e127-49f6-be89-64e09f4ef204-0')

## 2. Callback

In [18]:
# ! pip install httpx_sse

In [20]:
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate

class MyCustomHandler(BaseCallbackHandler):
    # Every time there is a new token, you will call the function in it.
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"My custom handler, token: {token}")


prompt = ChatPromptTemplate.from_messages(["Please tell me a joke about {animal}"])


llm = ChatZhipuAI(
    zhipuai_api_key = api_key,
    model="glm-4",
    max_tokens=1024,
    temperature=0.8, 
    streaming=True,
    callbacks=[MyCustomHandler()],
)


chain = prompt | llm

response = chain.invoke({"animal": "bear"})

My custom handler, token: Why
My custom handler, token:  don
My custom handler, token: 't
My custom handler, token:  bears
My custom handler, token:  like
My custom handler, token:  to
My custom handler, token:  use
My custom handler, token:  computers
My custom handler, token: ?

Because
My custom handler, token:  they
My custom handler, token:  prefer
My custom handler, token:  to
My custom handler, token:  keep
My custom handler, token:  it
My custom handler, token:  wild
My custom handler, token:  and
My custom handler, token:  un
My custom handler, token: -b
My custom handler, token: ear
My custom handler, token: -
My custom handler, token: able
My custom handler, token: !
My custom handler, token:  😄


In [22]:
# Langchain official streamingstdoutcallback
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = ChatZhipuAI(
    zhipuai_api_key = api_key,
    model="glm-4",
    max_tokens=1024,
    temperature=0.8, 
    streaming=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)


chain = prompt | llm

response = chain.invoke({"animal": "bear"})

/tmp/ipykernel_76278/1271503365.py:5: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = ChatZhipuAI(


Why don't bears like to use computers?

Because they prefer to keep things au naturel!

## 3. Memory

In [27]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")
memory.chat_memory.add_user_message("Hi")
memory.chat_memory.add_ai_message("Hi, how can I help you?")

In [28]:
memory.load_memory_variables({})

{'chat_history': 'Human: Hi\nAI: Hi, how can I help you?'}

In [37]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a chatbot who knows the chat_history"
        ),
 
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

chat = ChatZhipuAI(
    zhipuai_api_key = api_key,
    model="glm-4",
)

In [49]:
from langchain.chains import LLMChain

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chain = LLMChain(
    llm=chat,
    prompt=prompt,
    verbose=True,
    memory=memory
)




In [50]:
response = chain.invoke({"question": "Hi"})
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot who knows the chat_history
Human: Hi

> Finished chain.
{'question': 'Hi', 'chat_history': [HumanMessage(content='Hi'), AIMessage(content='### Conversation Summary:\n\nThe user greeted the chatbot. The chatbot acknowledged the greeting.\n\n---\n\nIf you have any questions or need further assistance, feel free to ask!')], 'text': '### Conversation Summary:\n\nThe user greeted the chatbot. The chatbot acknowledged the greeting.\n\n---\n\nIf you have any questions or need further assistance, feel free to ask!'}


In [51]:
response = chain.invoke({"question": "today is raining?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot who knows the chat_history
Human: Hi
AI: ### Conversation Summary:

The user greeted the chatbot. The chatbot acknowledged the greeting.

---

If you have any questions or need further assistance, feel free to ask!
Human: today is raining?

> Finished chain.


In [52]:
response = chain.invoke({"question": "Can I go out to play"})
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot who knows the chat_history
Human: Hi
AI: ### Conversation Summary:

The user greeted the chatbot. The chatbot acknowledged the greeting.

---

If you have any questions or need further assistance, feel free to ask!
Human: today is raining?
AI: As an AI, I don't have real-time data or the ability to check the weather. To find out if it's raining where you are, you can check a weather app or website, or simply look out the window. If you tell me your location, I can provide you with a general idea of the current weather based on some online sources, but it won't be as accurate as a dedicated weather service.
Human: Can I go out to play

> Finished chain.
{'question': 'Can I go out to play', 'chat_history': [HumanMessage(content='Hi'), AIMessage(content='### Conversation Summary:\n\nThe user greeted the chatbot. The chatbot acknowledged the greeting.\n\n---\n\nIf you have any questions or need further as